# This document explains how to use Rasterio to read spatial data files.
More detailed example can be found in [rasterio docs](https://rasterio.readthedocs.io/en/latest/quickstart.html#python-quickstart).

Import rasterio to begin.

In [4]:
import rasterio

Define data directory.

In [12]:
import os
data_dir = f'{os.getenv("HOME")}/university/s2_images'
os.listdir(data_dir)

['20180124_d2da9993-e10d-4aac-af5c-f160764c787d_ndvi.tif',
 '20180127_8a122447-3084-4705-b93a-323fa58cc655_ndvi.tif',
 '20180124_d2da9993-e10d-4aac-af5c-f160764c787d_ndvi_color.tif',
 '20180308_2995832d-5633-46b3-99c2-3a660df1c96b_ndvi_color.tif',
 '20180127_8a122447-3084-4705-b93a-323fa58cc655_rgb.tif',
 '20180114_a545f123-9f9b-4274-b2bd-8c34fb226234_ndvi_color.tif',
 '20180305_bbaf32e1-a995-41fb-bb83-71f90c08a6c8_ndvi_color.tif',
 '20180119_2b0b2695-c5cd-4bb9-845b-55240acfe779_ndvi.tif',
 '20180119_2b0b2695-c5cd-4bb9-845b-55240acfe779_ndvi_color.tif',
 '20180119_2b0b2695-c5cd-4bb9-845b-55240acfe779_rgb.tif',
 '20180114_a545f123-9f9b-4274-b2bd-8c34fb226234_ndvi.tif',
 '20180305_7721ec93-ce4b-4274-9490-728b84894fb7_rgb.tif',
 '20180114_a545f123-9f9b-4274-b2bd-8c34fb226234_rgb.tif',
 '20180127_8a122447-3084-4705-b93a-323fa58cc655_ndvi_color.tif',
 '20180124_6af9c6ad-3ea8-41c6-ab0f-75a94b2e5637_rgb.tif',
 '20180305_bbaf32e1-a995-41fb-bb83-71f90c08a6c8_rgb.tif',
 '20180325_ff3663a8-5def-4

Choose file.

In [15]:
file_name = '20180305_bbaf32e1-a995-41fb-bb83-71f90c08a6c8_rgb.tif'
file_path = os.path.join(data_dir,file_name)

Next, open the file.

In [17]:
dataset = rasterio.open(file_path)

In [18]:
dataset.name

'/home/ruslan/university/s2_images/20180305_bbaf32e1-a995-41fb-bb83-71f90c08a6c8_rgb.tif'

In [19]:
dataset.mode

'r'

In [20]:
dataset.closed

False

Properties of the raster data stored in the example GeoTIFF can be accessed through attributes of the opened dataset object. Dataset objects have bands and this example has a band count of 3

In [21]:
dataset.count

3

In [22]:
dataset.width

10980

In [23]:
dataset.height

10980

Some dataset attributes expose the properties of all dataset bands via a tuple of values, one per band. To get a mapping of band indexes to variable data types, apply a dictionary comprehension

In [24]:
{i: dtype for i, dtype in zip(dataset.indexes, dataset.dtypes)}

{1: 'uint8', 2: 'uint8', 3: 'uint8'}

## Dataset georeferencing
A GIS raster dataset is different from an ordinary image; its elements (or “pixels”) are mapped to regions on the earth’s surface. Every pixels of a dataset is contained within a spatial bounding box.

In [25]:
dataset.bounds

BoundingBox(left=699960.0, bottom=5290200.0, right=809760.0, top=5400000.0)

The value of `bounds` attribute is derived from a more fundamental attribute: the dataset’s geospatial transform.

In [26]:
dataset.transform

Affine(10.0, 0.0, 699960.0,
       0.0, -10.0, 5400000.0)

A dataset’s transform is an affine transformation matrix that maps pixel locations in (row, col) coordinates to (x, y) spatial positions. The product of this matrix and (0, 0), the row and column coordinates of the upper left corner of the dataset, is the spatial position of the upper left corner.

In [27]:
dataset.transform * (0, 0)

(699960.0, 5400000.0)

The position of the lower right corner is obtained similarly.

In [28]:
dataset.transform * (dataset.width, dataset.height)

(809760.0, 5290200.0)

But what do these numbers mean? These coordinate values are relative to the origin of the dataset’s coordinate reference system (CRS).

In [29]:
dataset.crs

CRS.from_dict(init='epsg:32634')

## Reading raster data

Data from a raster band can be accessed by the band’s index number. Following the GDAL convention, bands are indexed from 1.

In [30]:
dataset.indexes

(1, 2, 3)

In [31]:
band1 = dataset.read(1)

In [32]:
band1

array([[228, 255, 255, ..., 255, 255, 255],
       [198, 136, 141, ..., 255, 255, 255],
       [125, 108,  87, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ...,  25,  29,  27],
       [255, 255, 255, ...,  29,  30,  28],
       [  0, 255, 255, ...,  33,  31,  31]], dtype=uint8)

## Spatial indexing
Datasets have an `index()` method for getting the array indices corresponding to points in georeferenced space. To get the value for the pixel 100 kilometers east and 50 kilometers south of the dataset’s upper left corner, do the following.

In [37]:
x, y = (dataset.bounds.left + 100000, dataset.bounds.top - 50000)
x,y

(799960.0, 5350000.0)

In [38]:
row, col = dataset.index(x, y)

In [39]:
row, col

(5000, 10000)

In [40]:
band1[row, col]

255

To get the spatial coordinates of a pixel, use the dataset’s `xy()` method. The coordinates of the center of the image can be computed like this.

In [41]:
dataset.xy(dataset.height // 2, dataset.width // 2)

(754865.0, 5345095.0)

In [43]:
dataset.close()

In [44]:
dataset.closed

True